In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
#import seaborn as sns
import matplotlib.pyplot as plt
import os
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import csr_matrix, hstack
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import StratifiedKFold
from sklearn.metrics import log_loss

In [2]:
datadir = 'input/'
gatrain = pd.read_csv(os.path.join(datadir,'gender_age_train.csv'),
                      index_col='device_id')
gatest = pd.read_csv(os.path.join(datadir,'gender_age_test.csv'),
                     index_col = 'device_id')
phone = pd.read_csv(os.path.join(datadir,'phone_brand_device_model.csv'))
# Get rid of duplicate device ids in phone
phone = phone.drop_duplicates('device_id',keep='first').set_index('device_id')
events = pd.read_csv(os.path.join(datadir,'events.csv'),
                     parse_dates=['timestamp'], index_col='event_id')
appevents = pd.read_csv(os.path.join(datadir,'app_events.csv'), 
                        usecols=['event_id','app_id','is_active'],
                        dtype={'is_active':bool})
applabels = pd.read_csv(os.path.join(datadir,'app_labels.csv'))
label_cat = pd.read_csv(os.path.join(datadir,'label_categories.csv'))

In [3]:
a = appevents.merge(events[['device_id','timestamp','longitude','latitude']], how='left',left_on='event_id',right_index=True)

In [4]:
a.isnull().sum()

event_id     0
app_id       0
is_active    0
device_id    0
timestamp    0
longitude    0
latitude     0
dtype: int64

In [5]:
a.head()

,event_id,app_id,is_active,device_id,timestamp,longitude,latitude
0,2,5927333115845830913,True,-6401643145415154744,2016-05-01 00:54:12,103.65,30.97
1,2,-5720078949152207372,False,-6401643145415154744,2016-05-01 00:54:12,103.65,30.97
2,2,-1633887856876571208,False,-6401643145415154744,2016-05-01 00:54:12,103.65,30.97
3,2,-653184325010919369,True,-6401643145415154744,2016-05-01 00:54:12,103.65,30.97
4,2,8693964245073640147,True,-6401643145415154744,2016-05-01 00:54:12,103.65,30.97


In [6]:
labelsss = applabels.merge(label_cat,how='left',on='label_id')

In [7]:
labelsss.head()

,app_id,label_id,category
0,7324884708820027918,251,Finance
1,-4494216993218550286,251,Finance
2,6058196446775239644,406,unknown
3,6058196446775239644,407,DS_P2P net loan
4,8694625920731541625,406,unknown


In [8]:
id_grp = pd.DataFrame(labelsss.groupby(['app_id'])['label_id'].unique())

In [9]:
category_grp = pd.DataFrame(labelsss.groupby(['app_id'])['category'].unique())

In [10]:
id_grp.head()

,label_id
app_id,
-9223281467940916832,"[796, 795, 794, 405]"
-9222877069545393219,[135]
-9222785464897897681,"[812, 795, 794, 405]"
-9222198347540756780,"[810, 795, 794, 405]"
-9221970424041518544,"[714, 704, 548, 813, 795, 794, 405]"


In [11]:
category_grp.head()

,category
app_id,
-9223281467940916832,"[Cards RPG, game, Tencent, Custom label]"
-9222877069545393219,[education outside class]
-9222785464897897681,"[Parkour avoid class, game, Tencent, Custom la..."
-9222198347540756780,"[Casual puzzle categories, game, Tencent, Cust..."
-9221970424041518544,"[1 free, Property Industry 2.0, Industry tag, ..."


In [12]:
id_grp['category'] = category_grp['category']

In [13]:
cnt = pd.DataFrame(labelsss.groupby(['app_id'])['label_id'].agg(['count']))

In [15]:
id_grp['labels_count'] = cnt['count']

In [17]:
id_grp.head()

,label_id,category,labels_count
app_id,,,
-9223281467940916832,"[796, 795, 794, 405]","[Cards RPG, game, Tencent, Custom label]",4
-9222877069545393219,[135],[education outside class],1
-9222785464897897681,"[812, 795, 794, 405]","[Parkour avoid class, game, Tencent, Custom la...",4
-9222198347540756780,"[810, 795, 794, 405]","[Casual puzzle categories, game, Tencent, Cust...",4
-9221970424041518544,"[714, 704, 548, 813, 795, 794, 405]","[1 free, Property Industry 2.0, Industry tag, ...",7


In [18]:
a.head()

,event_id,app_id,is_active,device_id,timestamp,longitude,latitude
0,2,5927333115845830913,True,-6401643145415154744,2016-05-01 00:54:12,103.65,30.97
1,2,-5720078949152207372,False,-6401643145415154744,2016-05-01 00:54:12,103.65,30.97
2,2,-1633887856876571208,False,-6401643145415154744,2016-05-01 00:54:12,103.65,30.97
3,2,-653184325010919369,True,-6401643145415154744,2016-05-01 00:54:12,103.65,30.97
4,2,8693964245073640147,True,-6401643145415154744,2016-05-01 00:54:12,103.65,30.97


In [19]:
df_final = a.merge(id_grp[['label_id','category','labels_count']], how='left',left_on='app_id',right_index=True)

In [20]:
df_final.shape

(32473067, 10)

In [21]:
df_final.head()

,event_id,app_id,is_active,device_id,timestamp,longitude,latitude,label_id,category,labels_count
0,2,5927333115845830913,True,-6401643145415154744,2016-05-01 00:54:12,103.65,30.97,"[549, 710, 704, 548, 172]","[Property Industry 1.0, Relatives 1, Property ...",5
1,2,-5720078949152207372,False,-6401643145415154744,2016-05-01 00:54:12,103.65,30.97,"[721, 704, 548, 302, 303]","[Personal Effectiveness 1, Property Industry 2...",5
2,2,-1633887856876571208,False,-6401643145415154744,2016-05-01 00:54:12,103.65,30.97,"[251, 263, 306, 302, 405, 730, 756, 757, 775, ...","[Finance, Debit and credit, unknown, Custom la...",12
3,2,-653184325010919369,True,-6401643145415154744,2016-05-01 00:54:12,103.65,30.97,"[405, 730, 747, 749, 776, 782, 785, 255, 251, ...","[Custom label, And the Church, Insurance, Lowe...",10
4,2,8693964245073640147,True,-6401643145415154744,2016-05-01 00:54:12,103.65,30.97,"[549, 710, 704, 548, 172]","[Property Industry 1.0, Relatives 1, Property ...",5


In [22]:
df_final.to_hdf('events_data.hdf5','table')

/home/Lenovo/anaconda3/lib/python3.5/site-packages/pandas/core/generic.py:1101: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block4_values] [items->['label_id', 'category']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)
